Ce code sert à transformer les masques en bounding boxes

In [1]:
import os
import cv2
import csv
import numpy as np

# Dossier contenant les images PNG
input_folder = "sequences_bis"
output_csv = "bounding_boxes_all.csv"
EXPAND_RATIO = 1.1  # Augmenter de 10% de chaque côté

def get_bounding_box(image_path):
    """Retourne la bounding box du masque blanc sur fond noir avec un léger agrandissement."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None  # Aucun objet détecté

    x_min, y_min, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    x_max, y_max = x_min + w, y_min + h

    # Agrandir la bbox
    x_center, y_center = (x_min + x_max) // 2, (y_min + y_max) // 2
    new_w, new_h = int(w * EXPAND_RATIO), int(h * EXPAND_RATIO)

    x_min = max(0, x_center - new_w // 2)  # Évite les valeurs négatives
    y_min = max(0, y_center - new_h // 2)
    x_max = x_min + new_w
    y_max = y_min + new_h

    return (x_min, y_min, x_max, y_max)


# Lire toutes les images du dossier et extraire les bounding boxes
data = []
for filename in sorted(os.listdir(input_folder)):  # Trier pour consistance
    if filename.endswith(".png"):
        image_path = os.path.join(input_folder, filename)
        bbox = get_bounding_box(image_path)
        if bbox:
            data.append([filename, bbox])

# Sauvegarde dans un CSV
with open(output_csv, "w", newline="") as f:
    writer = csv.writer(f)
    for row in data:
        writer.writerow(row)

print(f"CSV généré: {output_csv}")


CSV généré: bounding_boxes_all.csv


In [ ]:

def display_first_image_with_bbox():
    """Affiche la première image avec sa bounding box."""
    for filename in sorted(os.listdir(input_folder)):  # Trier pour consistance
        if filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            bbox = get_bounding_box(image_path)
            if bbox:
                image = cv2.imread(image_path)  # Charger en couleur pour affichage
                x_min, y_min, x_max, y_max = bbox
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Dessiner la BBox
                cv2.imshow(f"Bounding Box - {filename}", image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                break 

display_first_image_with_bbox()
